Import Packages

In [145]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
import seaborn as sns

sns.set_style('darkgrid')
%matplotlib inline

import requests
from PIL import Image
import shutil
import urllib
from io import BytesIO
import base64

from os.path import exists

from reportlab.lib.pagesizes import LETTER
from reportlab.lib.units import inch
from reportlab.pdfgen.canvas import Canvas
from reportlab.platypus import SimpleDocTemplate, ListFlowable, ListItem, Table, TableStyle
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import Paragraph
from reportlab.platypus import Image as IMG
from reportlab.lib import colors

Import Dataset

In [146]:
df_raw = pd.read_excel("RawData.xlsx");
print(df_raw.info());
df_raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ID                    2 non-null      object 
 1   Name                  2 non-null      object 
 2   Capsule Image         2 non-null      object 
 3   Sleeve Image          2 non-null      object 
 4   Type                  2 non-null      object 
 5   Status                2 non-null      object 
 6   Category              2 non-null      object 
 7   Cup Size              2 non-null      object 
 8   Headline              2 non-null      object 
 9   Intensity             2 non-null      int64  
 10  Price                 2 non-null      float64
 11  Notes                 2 non-null      object 
 12  Taste                 2 non-null      object 
 13  Acidity               2 non-null      int64  
 14  Bitterness            2 non-null      int64  
 15  Roastiness            2 non

,ID,Name,Capsule Image,Sleeve Image,Type,Status,Category,Cup Size,Headline,Intensity,...,Acidity,Bitterness,Roastiness,Body,Description,Origin,Roasting,Contents & Allergens,Ingredients,Net Weight
0,OL1,Ispirazione Napoli,https://www.nespresso.com/ecom/medias/sys_mast...,https://www.nespresso.com/shared_res/agility/n...,Original,Current,Italian Espressos,25 ml (Ristretto) & 40 ml (Espresso),Dark & Creamy,13,...,1,5,5,5,"Ispirazione Napoli pays tribute to the short, ...",This intense blend is created mostly using Rob...,"A dark and intense roast, done with carefully ...",10 capsules of roast and ground coffee for the...,Roast and ground coffee,57 g – 2.01 oz for 10 capsules
1,OL2,Kazaar,https://www.nespresso.com/ecom/medias/sys_mast...,https://m.media-amazon.com/images/I/51vv3BK5ST...,Original,Current,Italian Espressos,25 ml (Ristretto) & 40 ml (Espresso),Exceptionally Intense & Syrupy,12,...,5,1,5,5,It reflects both the Palermo coastal lifestyle...,Together the Arabica and Robusta balance each ...,"A long, dark roast inspired by the union of Si...",10 capsules of roast and ground coffee for the...,Roast and ground coffee,60 g – 2.11 oz for 10 capsules


Download Images

In [147]:
def download_image(dataframe, index, folder):
    ID = dataframe.loc[index, "ID"];
    Name = dataframe.loc[index, "Name"];
    Type = dataframe.loc[index, "Type"];
    
    success = True;

    filename_capsule = f"{folder}/{ID}_{Name}_{Type}_Capsule.png";
    filename_sleeve = f"{folder}/{ID}_{Name}_{Type}_Sleeve.png";

    # Check if images already exist in the Images directory
    if (exists(filename_capsule) == True) & (exists(filename_sleeve) == True):
        print(f"{ID}: {Type} {Name} coffee images already exist in the Images directory.");
    
        dataframe.loc[index, "Capsule Image"] = filename_capsule;
        dataframe.loc[index, "Sleeve Image"] = filename_sleeve;
    else:
        # Attempt to download images using the URLs provided in raw dataset
        try:
            img_capsule = Image.open(requests.get(dataframe.loc[index, "Capsule Image"], stream=True).raw);
            img_sleeve = Image.open(requests.get(dataframe.loc[index, "Sleeve Image"], stream=True).raw);
        except:
            success = False;
        # Indicate which coffee's images cannot be automatically downloaded
        if success == False:
            print(f"{ID}: {Type} {Name} (i.e., index = {index}) coffee images could not be downloaded automatically.");
        # Save coffee images to Images directory and adjust the column values for Capsule Image and Sleeve Image to match with the newly downloaded images' local filepath
        else:
            img_capsule.save(filename_capsule);
            img_sleeve.save(filename_sleeve);

            dataframe.loc[index, "Capsule Image"] = filename_capsule;
            dataframe.loc[index, "Sleeve Image"] = filename_sleeve;

            print(f"{ID}: {Type} {Name} coffee images downloaded & saved.");

Create Charts

In [148]:
def createTasteChart(dataframe, index, folder):
    ID = dataframe.loc[index, "ID"];
    Name = dataframe.loc[index, "Name"];
    Type = dataframe.loc[index, "Type"];

    title = f"{Type} - {Name}: Taste Profile";
    filename_chart = f"{folder}/{ID}_{Name}_{Type}_Taste Profile.png";

    if (exists(filename_chart) == True):
        print(f"{ID}: {Type} {Name} taste profile chart already exists in the Charts directory.");
    else:
        data = [];
        for col in ["Acidity","Bitterness","Roastiness","Body"]:
            data.append([col, dataframe.loc[index, col]]);
        df_agg = pd.DataFrame(data, columns=["Profile","Measure"]);

        fig, ax = plt.subplots(figsize=(8,4));
        ax = sns.barplot(data=df_agg, x="Profile", y="Measure");
        ax.set(title=title);

        plt.savefig(filename_chart);

        print(f"{ID}: {Type} {Name} taste profile chart saved to Charts directory.");
    
    dataframe.loc[index, "Taste Profile Chart"] = filename_chart;

Procedurally Create Images & Charts

In [149]:
df = df_raw.copy();
df["Taste Profile Chart"] = "";
for i in df.index:
    download_image(df, i, "Images");
    createTasteChart(df, i, "Charts");

OL1: Original Ispirazione Napoli coffee images already exist in the Images directory.
OL1: Original Ispirazione Napoli taste profile chart already exists in the Charts directory.
OL2: Original Kazaar coffee images already exist in the Images directory.
OL2: Original Kazaar taste profile chart already exists in the Charts directory.


Create Report

In [150]:
def add_image_to_report(image_path, width, height):
    img = IMG(image_path);
    img.drawWidth = width
    img.drawHeight = height
    return img;

In [151]:
def buildReport(dataframe, index):
    # Instantiate Report and calibrate formatting ***************************
    my_doc = SimpleDocTemplate(
        "Guides/" + dataframe.loc[i, "ID"] + "_" + dataframe.loc[i, "Name"] + ".pdf",
        pagesize=LETTER
    );

    sample_style_sheet = getSampleStyleSheet();
    sample_style_sheet.list();
    
    sample_style_sheet.add(ParagraphStyle(
        name='CustomParagraph', 
        parent=sample_style_sheet['BodyText'],
        spaceBefore=5,
        spaceAfter=10,
    ))

    sample_style_sheet.add(ParagraphStyle(
        name='CustomHeading', 
        parent=sample_style_sheet['Heading1'],
        spaceBefore=10,
        spaceAfter=10,
    ))

    table_style = TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.gray),  # Header background color
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.white),  # Header text color
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),  # Cell alignment
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),  # Header font
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),  # Header bottom padding
        ('BACKGROUND', (0, 1), (-1, -1), colors.lightgrey),  # Data background color
        ('GRID', (0, 0), (-1, -1), 1, colors.black),  # Gridlines
    ]);

    flowables = [];

    # Create Report Information *****************************************

    # Report Title (Capsule Image, Type, Name) -----------------------------

    # Capsule Image:
    img_capsule = add_image_to_report(dataframe.loc[i, "Capsule Image"], 100, 100);
    flowables.append(img_capsule);
    
    reportTitle = f"{dataframe.loc[index, 'Type']} - {dataframe.loc[index, 'Name']}";
    flowables.append(Paragraph(reportTitle, sample_style_sheet['Title']));

    # General Information (Status, Category, Cup Size, Headline, Intensity, Price, Notes) -----
    flowables.append(Paragraph("General Information", sample_style_sheet['Heading1']));

    genInfoList = [];

    for col in ["Status","Category","Cup Size","Headline", "Intensity","Price","Notes"]:
        if (isinstance(dataframe.loc[index, col], int) == True) | (isinstance(dataframe.loc[index, col], float) == True):
            genInfoList.append(f"<strong>{col}:</strong> " + str(dataframe.loc[index, col]));
        else:
            genInfoList.append(f"<strong>{col}:</strong> {dataframe.loc[index, col]}");
    
    for info in genInfoList:
        flowables.append(Paragraph(info, bulletText="-"));

    # Taste Information (Taste, Acidity, Bitterness, Roastiness, Body) ---------
    flowables.append(Paragraph("Taste Information", sample_style_sheet['CustomHeading']));

    flowables.append(Paragraph(dataframe.loc[index, "Taste"], sample_style_sheet['CustomParagraph']));

    img_tasteProfile = add_image_to_report(dataframe.loc[i, "Taste Profile Chart"], 500, 250);
    flowables.append(img_tasteProfile);

    # The Story (Sleeve Image, Description, Origin, Roasting) ----------------
    flowables.append(Paragraph("The Story", sample_style_sheet['Heading1']));

    # Sleeve Image:
    img_sleeve = add_image_to_report(dataframe.loc[i, "Sleeve Image"], 200, 100);
    flowables.append(img_sleeve);

    for col in ["Description","Origin","Roasting"]:
        if (dataframe.loc[index, col] != ""):
            flowables.append(Paragraph(dataframe.loc[index, col], sample_style_sheet['CustomParagraph']));

    # Additional Information (Contents & Allergens, Ingredients, Net Weight) ---
    flowables.append(Paragraph("Additional Information", sample_style_sheet['Heading1']));

    addInfoData = [
        ["Fact","Detail"]
    ];
    for col in ["Contents & Allergens","Ingredients","Net Weight"]:
        addInfoData.append([col, dataframe.loc[index, col]]);
    addInfoTable = Table(addInfoData);
    addInfoTable.setStyle(table_style);
    flowables.append(addInfoTable);

    # Generate Report *********************************************************

    try:
        my_doc.build(flowables);
        print("Report Successfully Generated");
    except:
        print("Error Generating Report");

In [152]:
for i in df.index:
    buildReport(df, i);

BodyText None
    name = BodyText
    parent = <ParagraphStyle 'Normal'>
    alignment = 0
    allowOrphans = 0
    allowWidows = 1
    backColor = None
    borderColor = None
    borderPadding = 0
    borderRadius = None
    borderWidth = 0
    bulletAnchor = start
    bulletFontName = Helvetica
    bulletFontSize = 10
    bulletIndent = 0
    embeddedHyphenation = 0
    endDots = None
    firstLineIndent = 0
    fontName = Helvetica
    fontSize = 10
    hyphenationLang = 
    justifyBreaks = 0
    justifyLastLine = 0
    leading = 12
    leftIndent = 0
    linkUnderline = 0
    rightIndent = 0
    spaceAfter = 0
    spaceBefore = 6
    spaceShrinkage = 0.05
    splitLongWords = 1
    strikeColor = None
    strikeGap = 1
    strikeOffset = 0.25*F
    strikeWidth = 
    textColor = Color(0,0,0,1)
    textTransform = None
    underlineColor = None
    underlineGap = 1
    underlineOffset = -0.125*F
    underlineWidth = 
    uriWasteReduce = 0
    wordWrap = None

Bullet bu
    name = B

Export Final Dataset

In [153]:
df.to_excel('Data.xlsx', index=False);